##2. Etapa de preprocesado de texto
El alumno preparará una etapa de preprocesado de reviews que permita adecuar
el formato de las mismas a uno más adecuado. Será la etapa previa al entrenamiento del
modelo de sentimiento.
Todo el preprocesado deberá incluirse en una función de Python que contenga
todo el procesado de texto. Esta función puede (es recomendable) contener otras funciones
que realicen tareas más concretas (eliminar stopwords, eliminar signos de puntuación, etc.).

In [ ]:
import spacy

def preparar_dataframe_reviews(
    df,
    columna_texto='reviews.text',
    columna_rating='reviews.rating',
    stopwords_adicionales=None,
    modelo_spacy='en_core_web_sm',
    sentimiento_negativo=(1, 2, 3),
    sentimiento_positivo=(4, 5),
    columnas_finales=('clean_review', 'sentiment')
):
    """
    Preprocesa el dataframe:
    - Limpia y lematiza el texto (spaCy)
    - Convierte rating a binario (0: negativo, 1: positivo)
    - Deja solo columnas finales

    Parámetros:
        df: DataFrame original
        columna_texto: nombre de la columna con el texto original
        columna_rating: nombre de la columna con el rating
        stopwords_adicionales: set/lista de stopwords de dominio
        modelo_spacy: modelo spaCy
        sentimiento_negativo: tupla/lista con ratings a considerar negativos
        sentimiento_positivo: tupla/lista con ratings a considerar positivos
        columnas_finales: columnas a dejar al final
    Devuelve:
        df_procesado: DataFrame limpio y listo
    """
    # 1. Limpieza de nulos y tipos
    df = df[[columna_texto, columna_rating]].dropna()
    df[columna_rating] = df[columna_rating].astype(int)

    # 2. Preprocesamiento de texto
    def procesar(texto):
        return preprocesar_texto_spacy(
            texto,
            stopwords_adicionales=stopwords_adicionales,
            modelo=modelo_spacy
        )
    df['clean_review'] = df[columna_texto].apply(procesar)

    # 3. Conversión de rating a sentimiento binario
    def rating_a_sentimiento(rating):
        if rating in sentimiento_positivo:
            return 1
        else:
            return 0
    df['sentiment'] = df[columna_rating].apply(rating_a_sentimiento)

    # 4. Filtrar solo columnas finales
    df_final = df[list(columnas_finales)].copy()
    df_final = df_final.dropna()

    return df_final.reset_index(drop=True)
